<a href="https://colab.research.google.com/github/birajsilwal/TweetSentiment/blob/master/TweetSentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import re

# Filter tweets class

In [50]:

'''
this class filters (removes tweet tags, punctuations,
irrelevant words such as a, an, the) from the tweets
and returns array of clean filtered tweets
'''
class FilterTweets:

    
    def filterTweets(self, file_path):
      final_tweets = []
      tweets = []
      tweets_raw = open(file_path)
      for tweet in tweets_raw:
          tweet = tweet.lower()
          tweet = re.sub(":|;|\(|\)|!|\'s|\"|\.|n\'t|,|\?|i\'m|i\'ve", "", tweet)
          tweet = re.sub(" an | the | is | of | a | i | was | and |"
                          " on | in | off | all | it | me | you | to |"
                          " into | we | your | that | they | can | could |"
                          " should | do | does | for | my | at | so |"
                          " if | has | have | had | from | such | are |"
                          " not | this | now | but | go | day |"
                          "-|_| up | down | these | today | lol |"
                          " lmao | af | get | got | here | there | who |"
                          " what | am | no | why | with | us | our | bro |"
                          " too | then | ur | zero | ah | see | saw ", " ", tweet)
          tweets.append(tweet)


      # cleaning up twice because of space issue
      tweets1 = []
      for tweet in tweets:
          tweet = tweet.lower()
          tweet = re.sub(" an | the | is | of | a | i | was | and |"
                          " on | in | off | all | it | me | you | to |"
                          " into | we | your | that | they | can | could |"
                          " should | do | does | for | my | at | so |"
                          " if | has | have | had | from | such | are |"
                          " not | this | now | but | go | day |"
                          "-|_| up | down | these | today | lol |"
                          " lmao | af | get | got | here | there | who |"
                          " what | am | no | why | with | us | our | bro |"
                          " too | then | ur | zero | ah | see | saw ", " ", tweet)
          tweets1.append(tweet)


      '''
      if there contains a substring starting with http or @,
      remove that substring from the tweet
      '''
      for str in tweets1:
          index = 0
          string_arr = str.split()
          for string in string_arr:
              if 'http' in string or '@' in string:
                  string_arr[index] = string.replace(string, '')
              index += 1

          tweet = ' '
          tweet = tweet.join(string_arr)
          final_tweets.append(tweet)
          # print(tweet)
      return final_tweets

# Filtered datasets

In [51]:
filter_tweets = FilterTweets()

tweets_training_filtered = filter_tweets.filterTweets('/content/drive/MyDrive/Colab Notebooks/Tweets/noCR_train.txt')
print(len(tweets_training_filtered))
print(tweets_training_filtered[0])


tweets_testing_filtered = filter_tweets.filterTweets('/content/drive/MyDrive/Colab Notebooks/Tweets/noCR_test.txt')
print(len(tweets_testing_filtered))

10271
1 5sos calum5sos luke5sos ashton5sos hey guys please follow ✧｡ chelny ｡✧
1145


# Tweet class

In [52]:
class Tweet:
  def __init__(self, tweet, zero_or_one):
      self.tweet = tweet
      self.zero_or_one = zero_or_one
      self.sentiment = self.get_sentiment()

  
  def get_sentiment(self):
    if self.zero_or_one == '0':
      return "NEGATIVE"
    elif self.zero_or_one == '1':
      return "POSITIVE" 


In [53]:
# creating tweet objects for each tweet
tweets_training = []
tweets_testing = []

for tweet in tweets_training_filtered:
  string_arr = tweet.split()
  zero_or_one = string_arr.pop(0)

  tweet1 = ' '
  tweet1 = tweet1.join(string_arr)
  tweets_training.append(Tweet(tweet1, zero_or_one))

In [62]:
# x and y values for both training and testing dataset
train_x = [x.tweet for x in tweets_training]
train_y = [x.sentiment for x in tweets_training]

test_x = [x.tweet for x in tweets_testing]
test_y = [x.sentiment for x in tweets_testing]